In [1]:
using JuMP
using Gurobi
using MixedIntegerExperiments
using Polyhedra
using CDDLib
using Plots
using DrakeVisualizer
using AxisArrays
using BenchmarkTools

In [2]:
# Pkg.clone("https://github.com/andreasnoack/LinearAlgebra.jl")
# Pkg.clone("git@github.com:JuliaPolyhedra/ConvexHull.jl.git")

In [3]:
# Pkg.checkout("CDDLib", "eltypeit")
# Pkg.checkout("Polyhedra")

In [4]:
verbose = false;
show_plots = false;

In [5]:
# parameters
nsteps = 100
ncontacts = 2
h = 0.1
I = 1.
m = 10.
g = [0; -9.81]
r0 = [0.5; 0.8]
ṙ0 = [0.0; 0.0]
θ0 = 0.
ω0 = 0.
contactregion = contact_region([0.; 0.], [1.; 0.], 0.8, 2 * m * norm(g))
freeregion = axis_aligned_free_box_region([0.; 0.], [1.; 1.])
environment = [contactregion; freeregion];

In [6]:
if show_plots
    plt = plot()
    plot_polyhedron!(polyhedron(contactregion.position); lab = "contact", fillcolor = "black")
    plot_polyhedron!(polyhedron(freeregion.position); lab = "free", opacity = 0.5)
    title!(plt, "Environment")
    plt
end

In [7]:
if show_plots
    plt = plot()
    plot_polyhedron!(polyhedron(contactregion.force); opacity = 0.5, lab = "Contact region")
    title!("Allowable contact forces")
end

In [8]:
model = Model(solver = GurobiSolver(Presolve = 0, OutputFlag = Int(verbose)));

In [9]:
coords = Axis{:coord}([:x, :z])
steps = Axis{:step}(1 : nsteps)
contacts = Axis{:contact}(1 : ncontacts)
regions = Axis{:region}(1 : length(environment))

AxisArrays.Axis{:region,UnitRange{Int64}}(1:2)

In [10]:
rs = @axis_variables(model, r, coords, steps)
θs = @axis_variables(model, θ, steps)
ṙs = @axis_variables(model, ṙ, coords, steps)
ωs = @axis_variables(model, ω, steps)
rcs = @axis_variables(model, rc, coords, contacts, steps)
fs = @axis_variables(model, f, coords, contacts, steps)
Fs = @axis_variables(model, F, coords, steps)
Ts = @axis_variables(model, T, steps)
ss = @axis_variables(model, s, coords, contacts, steps)
wzxs = @axis_variables(model, wzx, contacts, steps)
wxzs = @axis_variables(model, wxz, contacts, steps)
γs = @axis_variables(model, γ, contacts, regions, steps(1 : length(steps) - 1))
Γs = @axis_variables(model, Γ, contacts, regions);

In [11]:
# 4.6
# TODO: CHECK CHECK CHECK CHECK CHECK
zs = AxisArray(Array{Variable, 3}(ncontacts, length(environment), nsteps), contacts, regions, steps)
product_regions = collect(region.position * region.force for region in environment)
for i = 1 : ncontacts, n = 1 : nsteps
    rc = rcs[contacts(i), steps(n)]
    f = fs[contacts(i), steps(n)]
    zs[contacts(i), steps(n)] = hull_reformulation(model, product_regions, [rc; f])
end

In [12]:
# (4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    s = Vector(ss[contacts(i), steps(n)])
    rc = Vector(rcs[contacts(i), steps(n)])
    r = Vector(rs[steps(n)])
    @constraint(model, s .== rc - r)
end

In [13]:
# 4.18
for n = 1 : nsteps - 1
    @constraint(model, Vector(rs[steps(n + 1)]) .== Vector(rs[steps(n)]) + h * Vector(ṙs[steps(n + 1)]))
    @constraint(model, θs[steps(n + 1)] == θs[steps(n)] + h * ωs[steps(n + 1)])
    @constraint(model, Vector(ṙs[steps(n + 1)]) .== Vector(ṙs[steps(n)]) + h / m * Vector(Fs[steps(n + 1)]))
    @constraint(model, ωs[steps(n + 1)] == ωs[steps(n)] + h / I * Ts[steps(n + 1)])
end

for n = 1 : nsteps
    @constraint(model, Vector(Fs[steps(n)]) .== m * g + sum(fs[steps(n), contacts(i)] for i = 1 : length(contacts)))
    @constraint(model, Ts[steps(n)] == sum(wzxs[steps(n), contacts(i)] - wxzs[steps(n), contacts(i)] for i = 1 : length(contacts)))
end

In [14]:
# convex bilinear term approximations (4.12, 4.13)
for i = 1 : ncontacts, n = 1 : nsteps
    # TODO: ranges, piecewise
    wxz = wxzs[contacts(i), steps(n)]
    wzx = wzxs[contacts(i), steps(n)]
    s = ss[contacts(i), steps(n)]
    f = fs[contacts(i), steps(n)]
    mccormick_envelope_constraints(model, wxz, s[:x], f[:z], -100., 100., -100., 100.)
    mccormick_envelope_constraints(model, wzx, s[:z], f[:x], -100., 100., -100., 100.)
end

In [15]:
# initial conditions
@constraints(model, begin
    Vector(rs[steps(1)]) .== r0
    θs[steps(1)] == θ0
    Vector(ṙs[steps(1)]) .== ṙ0
    ωs[steps(1)] == ω0
end)

# reachability; TODO: better version
@constraint(model, Vector(rs[coords(:x)]) .>= 0);
@constraint(model, Vector(rs[coords(:x)]) .<= 1);
@constraint(model, Vector(rs[coords(:z)]) .>= 0);
@constraint(model, Vector(rs[coords(:z)]) .<= 1);

In [16]:
# TODO: more task-specific constraints
# @constraints(model, begin
#     [n = 1 : nsteps], r[2, n] >= 0.
# end)

In [17]:
# section 4.4
# objective function + slack parameterization
@constraint(model, Array(Γs) .== sum(γs[steps(n)] for n = 1 : length(steps) - 1));
for n = 1 : length(steps) - 1
    @constraint(model, -Array(γs[steps(n)]) .<= Array(zs[steps(n + 1)] - zs[steps(n)]))
    @constraint(model, Array(zs[steps(n + 1)] - zs[steps(n)]) .<= Array(γs[steps(n)]))
    @constraint(model, 0 .<= Array(γs))
    @constraint(model, Array(γs) .<= 1)
end

@objective(model, Min, sum(Γ) + vecdot(fs, fs)) #= + vecdot(ṙcs, ṙcs)=#

:Min

In [18]:
@benchmark solve($model)

BenchmarkTools.Trial: 
  memory estimate:  4.22 MiB
  allocs estimate:  91
  --------------
  minimum time:     431.217 ms (0.00% GC)
  median time:      458.567 ms (0.00% GC)
  mean time:        457.152 ms (0.04% GC)
  maximum time:     475.872 ms (0.00% GC)
  --------------
  samples:          11
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [22]:
plot(getvalue(rs[:z]))

In [20]:
# vis = MixedIntegerExperiments.plot_piecewise_mccormick(10, -1., 1., -1., 1.);

In [21]:
# delete!(vis)